# Decision Trees and Random Forests

Welcome! In this notebook, we'll explore two powerful and interpretable machine learning models: **Decision Trees** and **Random Forests**.

We will cover:

- What decision trees are and how they split data
- Entropy, Information Gain, and Gini Index
- Overfitting and pruning
- Random forests and the idea of bagging
- Implementing models with `scikit-learn`
- Visualizing and interpreting models

Let's dive in!

### 🌳 Decision Trees
---

#### 📌 1. What is a Decision Tree?

A **Decision Tree** is a model that makes decisions by recursively splitting data based on feature values.

- **Root Node**: The first decision point.  
- **Internal Nodes**: Intermediate decisions based on features.  
- **Leaf Nodes**: Final output (e.g., class label or predicted value).

At each step, the goal is to choose a split that best separates the data into "pure" subsets — groups where most samples belong to the same class.

---

#### 🧮 2. Splitting Criteria

To build the tree, we need a way to measure how "impure" a group of data is, and how much a split reduces that impurity.

---

##### 📊 Entropy and Information Gain

**Entropy** measures uncertainty or disorder in a dataset:

$$
\text{Entropy}(S) = -\sum_{i=1}^{C} p_i \log_2 p_i
$$

Where:  
- $S$ is the dataset at a node  
- $C$ is the number of classes  
- $p_i$ is the proportion of samples in class $i$

**Examples**:  
- Perfectly mixed classes (e.g., 50/50): $\text{Entropy} = 1$  
- Pure class (all one label): $\text{Entropy} = 0$

---

**Information Gain (IG)** measures how much a feature reduces entropy:

$$
IG(S, A) = \text{Entropy}(S) - \sum_{v \in \text{Values}(A)} \frac{|S_v|}{|S|} \cdot \text{Entropy}(S_v)
$$

Where:  
- $A$ is a feature to split on  
- $\text{Values}(A)$ are the unique values of feature $A$  
- $S_v$ is the subset of $S$ where feature $A = v$

---

##### 🌀 Gini Impurity

An alternative to entropy is the **Gini Impurity**:

$$
\text{Gini}(S) = 1 - \sum_{i=1}^{C} p_i^2
$$

Gini is often used in practice (e.g., it's the default in `scikit-learn`) because it's simpler to compute and works well in many cases.

---

#### 🪓 3. Splitting Process

At each node:
1. Evaluate all features (and possible thresholds for numerical features).
2. For each, compute the entropy or Gini impurity of the resulting splits.
3. Choose the feature and threshold that maximize Information Gain (or minimize impurity).
4. Recursively split until a stopping condition is met (e.g., maximum depth, minimum samples per leaf, or pure leaf).

---

#### ⚠️ 4. Overfitting and Pruning

Decision Trees can easily overfit the training data.

**How to control overfitting:**
- Limit the **maximum depth** of the tree
- Set a **minimum number of samples** required to split a node
- Use **pruning** (remove branches with low predictive power)

---

#### ✅ 5. Advantages and Disadvantages

**Advantages:**
- Easy to understand and interpret
- Handles numerical and categorical data
- No feature scaling required
- Fast training and prediction

**Disadvantages:**
- Prone to overfitting
- Small changes in data can lead to very different trees
- Less accurate than ensemble methods like Random Forests



### 🌟 Worked Example: Splitting with Entropy and Information Gain

Let's say we have a small dataset of 10 students with the following features:

| Student | Studies? | Passes Exam |
|---------|----------|--------------|
| 1       | Yes      | Yes          |
| 2       | Yes      | Yes          |
| 3       | No       | No           |
| 4       | Yes      | Yes          |
| 5       | No       | No           |
| 6       | No       | No           |
| 7       | Yes      | Yes          |
| 8       | No       | No           |
| 9       | Yes      | Yes          |
| 10      | No       | No           |

We want to predict whether a student **passes the exam**, and we’re considering splitting the data based on whether they **study**.

---

#### Step 1: Calculate Entropy of the Full Dataset

There are:
- 5 students who **pass** (Yes)
- 5 students who **fail** (No)

So the entropy of the whole dataset is:

$$
Entropy(S) = -p_+ \log_2 p_+ - p_- \log_2 p_-
$$

Where:
- $p_+ = 5/10 = 0.5$
- $p_- = 5/10 = 0.5$

So:

$$
Entropy(S) = -0.5 \log_2(0.5) - 0.5 \log_2(0.5) = 1
$$

---

#### Step 2: Split the Data on "Studies?"

- Students who **study (Yes)**: 5 students → All passed
- Students who **don’t study (No)**: 5 students → All failed

So we split into two groups:

**Group 1 (Studies = Yes):**  
- 5 students  
- 5 pass, 0 fail → pure group  
- Entropy = 0

**Group 2 (Studies = No):**  
- 5 students  
- 0 pass, 5 fail → pure group  
- Entropy = 0

---

#### Step 3: Calculate Weighted Average Entropy After Split

$$
Entropy_{after} = \frac{5}{10} \cdot 0 + \frac{5}{10} \cdot 0 = 0
$$

---

#### Step 4: Calculate Information Gain

$$
IG = Entropy(S) - Entropy_{after} = 1 - 0 = 1
$$

So, splitting on "Studies?" gives us **perfect information gain**.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Load Iris dataset
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

In [ ]:
print(iris.DESCR)

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Decision Tree
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

### 📏 Precision, Recall, and F1-Score

In a binary classification setting, we define:

- **True Positives (TP)**: Model correctly predicts positive class  
- **False Positives (FP)**: Model incorrectly predicts positive class  
- **False Negatives (FN)**: Model misses the positive class  
- **True Negatives (TN)**: Model correctly predicts negative class

---

#### 🎯 Precision

Precision tells us **how many of the predicted positives were actually correct**.

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

---

#### 🎯 Recall (Sensitivity or True Positive Rate)

Recall tells us **how many of the actual positives were captured** by the model.

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

---

#### 🎯 F1-Score

F1-score is the **harmonic mean** of precision and recall. It balances the two metrics.

$$
F1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

---

These metrics are especially useful in **imbalanced classification problems**, where accuracy can be misleading.

#### 🟦 Macro Average

- **Macro avg** computes the metric **independently for each class**, and then **takes the unweighted mean**.
- **All classes are treated equally**, regardless of how many samples each class has.

**Example formula for Macro Precision:**

$$
\text{Precision}_{macro} = \frac{1}{C} \sum_{i=1}^{C} \text{Precision}_i
$$

Where:
- $C$ is the number of classes
- $\text{Precision}_i$ is the precision for class $i$

Same idea applies to recall and F1-score.

✅ Use macro avg if you care equally about **all classes**, even rare ones.

---

#### 🟨 Weighted Average

- **Weighted avg** computes the metric for each class and **weights it by the number of true instances** (support) in that class.
- Classes with more samples have more influence on the final score.

**Example formula for Weighted Precision:**

$$
\text{Precision}_{weighted} = \sum_{i=1}^{C} \frac{n_i}{N} \cdot \text{Precision}_i
$$

Where:
- $n_i$ = number of true samples in class $i$
- $N$ = total number of samples
- $\text{Precision}_i$ = precision for class $i$

✅ Use weighted avg when you want a score that reflects **class imbalance**.

---

#### Summary

| Metric Type   | Treats Classes Equally? | Accounts for Class Imbalance? |
|---------------|-------------------------|-------------------------------|
| Macro Average | ✅ Yes                  | ❌ No                         |
| Weighted Avg  | ❌ No                   | ✅ Yes                        |

In [ ]:
# Visualizing the tree
plt.figure(figsize=(12,8))
plot_tree(clf, feature_names=iris.feature_names, class_names=iris.target_names, filled=True)
plt.title("Decision Tree Visualization")
plt.show()

In [ ]:
# Load the Iris dataset
iris = load_iris()
X = iris.data[:, :2]  # Use only sepal length and sepal width for 2D plot
y = iris.target
target_names = iris.target_names

# Train a Decision Tree
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X, y)

# Plot decision boundaries
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

# 🧠 Predict class for each point in mesh
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# 🎨 Plot the regions
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)

# Plot training points
for i, class_name in enumerate(target_names):
    plt.scatter(X[y == i, 0], X[y == i, 1], label=class_name, edgecolor='k', s=50)

plt.xlabel('Sepal length (cm)')
plt.ylabel('Sepal width (cm)')
plt.title('Decision Tree Decision Regions (Iris Dataset)')
plt.legend()
plt.grid(True)
plt.show()

### 🌲 Random Forests

---

#### 📌 **What is a Random Forest?**

A **Random Forest** is an ensemble learning method that combines multiple **decision trees** to create a stronger and more accurate model. The key idea behind Random Forests is that by averaging many trees, the overall model becomes more robust and generalizes better than a single decision tree, which may overfit the data.

- **Ensemble Learning**: This refers to combining the predictions of multiple models to improve the overall performance.
- **Randomness**: Random Forests introduce randomness both in the selection of the training data and in the features used to build individual trees.

---

#### 🧠 **How Does a Random Forest Work?**

1. **Bootstrap Sampling**:  
   Random Forests build each tree using a different random subset of the training data. This subset is created by **bootstrap sampling** — randomly selecting data points with replacement. As a result, each tree is trained on slightly different data.

2. **Random Feature Selection**:  
   When building each decision tree, Random Forests don't consider all features for each split. Instead, they randomly select a subset of features at each node to decide the best split. This introduces additional diversity among the trees, making the forest stronger and reducing overfitting.

3. **Building Many Decision Trees**:  
   The model trains a large number of decision trees, each on a different subset of data and features. Each tree makes independent predictions based on the data it sees.

4. **Averaging or Voting**:  
   For **regression tasks**, Random Forests average the predictions of all the trees.  
   For **classification tasks**, Random Forests use majority voting, where the class predicted by the majority of trees becomes the final prediction.

---

#### 📊 **Key Concepts in Random Forests**

- **Bagging (Bootstrap Aggregating)**:  
  Bagging refers to the method of training each tree on a random subset of the data. The key idea is that training on different data subsets reduces the variance of the model without increasing bias too much.
  
- **Out-of-Bag (OOB) Error**:  
  For each tree, some data points are not selected during the bootstrap sampling. These points are called **out-of-bag** samples. The performance of the Random Forest can be evaluated using these OOB samples, which serve as a validation set for each tree.

- **Feature Randomness**:  
  At each node of the decision tree, only a random subset of features is considered for the best split. This ensures that the trees in the forest are diverse and reduces the risk of overfitting.

---

#### ⚡ **Advantages of Random Forests**

1. **Robustness**:  
   By combining many decision trees, Random Forests are less likely to overfit compared to a single decision tree. Even if some trees overfit, the random selection of data and features reduces the impact on the final model.

2. **Handling Missing Values**:  
   Random Forests are capable of handling missing values, especially if there are many trees in the forest. The algorithm can still make good predictions even when some data points are missing.

3. **Feature Importance**:  
   Random Forests can provide insights into which features are the most important in predicting the target variable. This is done by measuring how much each feature improves the split quality in the trees.

4. **Versatility**:  
   Random Forests can be used for both **classification** and **regression** tasks and are very flexible in handling different types of data, including numerical, categorical, and missing data.

---

#### ⚠️ **Disadvantages of Random Forests**

1. **Model Interpretability**:  
   Random Forests are often seen as a "black-box" model, meaning they are difficult to interpret. Unlike a single decision tree, which is easy to visualize and understand, the random forest's decision-making process is more complex.

2. **Computation Cost**:  
   Since Random Forests require the training of many decision trees, they can be computationally expensive and require more memory than individual decision trees.

3. **Slow Predictions**:  
   In some cases, predictions can be slower than using a single decision tree, especially when the number of trees in the forest is large.

---

#### 📈 **Summary**

- A **Random Forest** is an ensemble method built by training many decision trees on random subsets of the data and features.
- It combines **bagging** and **random feature selection** to create a robust model that performs well on a wide range of tasks.
- Random Forests offer strong generalization, high accuracy, and can handle missing data, but they come at the cost of model interpretability and computational resources.

---


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
# Load the Wine dataset
wine = load_wine()
X = wine.data
y = wine.target
feature_names = wine.feature_names
target_names = wine.target_names

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print(wine.DESCR)

In [ ]:
# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
clf.fit(X_train, y_train)

# Predict on test set
y_pred = clf.predict(X_test)

# Print classification report and confusion matrix
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=target_names))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Feature importances
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

# Display feature importances
print("\nFeature Importances:")
for i in indices:
    print(f"{feature_names[i]}: {importances[i]:.4f}")

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), [feature_names[i] for i in indices], rotation=90)
plt.ylabel("Importance")
plt.title("Feature Importances (Random Forest - Wine Dataset)")
plt.tight_layout()
plt.grid(True)
plt.show()

$$\text{Importance}(f) = \frac{1}{T} \sum_{t=1}^{T} \sum_{n \in I(f, t)} \Delta i_t^n
$$
- $T$ is the total number of trees in the forest
- $f$ is the feature for which importance is being calculated
- $I(f,t)$ is the set of nodes in tree $t$ where feature $f$ is used
- $\Delta i_T^n$ is the impurity decrease at node $n$ in tree $t$

$$\tilde{I}(f) = \frac{\text{Importance}(f)}{\sum_{f'} \text{Importance}(f')}
$$

## ⚡ Boosting
- Models are trained **sequentially**, with each trying to **fix the errors** of the previous one.
- Later models give **more weight to misclassified** points.
- Final prediction is a **weighted vote**.

We will use `AdaBoostClassifier` (short for **Adaptative Boosting**) to illustrate this method. 
- This method works sequentially, training each new model to focus more on the mistakes made by the previous ones.
- Each weak learner is assigned with a weight based on its performance.
- The final prediction is a weighted majority vote of all the learners.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [ ]:
# Load data from UCI (preloaded version from seaborn or you can download it externally)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", 
    "marital-status", "occupation", "relationship", "race", "sex", 
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)

In [ ]:
# Drop rows with missing values
data.dropna(inplace=True)

In [ ]:
data

In [ ]:
# Encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

In [ ]:
data

In [ ]:
# Split data
X = data.drop("income", axis=1)
y = data["income"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# AdaBoost model
boosting_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)

boosting_model.fit(X_train, y_train)
y_pred = boosting_model.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred, target_names=label_encoders['income'].classes_))